In [186]:
import random
import pandas as pd
import numpy as np
from dataclasses import dataclass, field
from itertools import product

In [187]:
DIM = 10
LST_LAYER = ["Reticle", "Prev1Reticle", "Prev2Reticle"]
LST_LAYER_NAME = ["L0", "L1", "L2"]
LST_SCALE = [1, 1, 150, 150, 150, 150, 15, 15, 15, 15]
LST_FEAT = ["Tx", "Ty", "Rot", "Nor", "Ex", "Ey", "RRot", "RMag", "ARRot", "ARMag"]
CHUCK_TYPE = 0

In [188]:
if CHUCK_TYPE == 0:
    LST_CHUCK = [["S1", "S2"]]
elif CHUCK_TYPE == 1:
    LST_CHUCK = [["BASE"]]
elif CHUCK_TYPE == 2:
    LST_CHUCK = [["S1", "S2"], ["BASE"]]

rng_part_name = np.random.default_rng()
rng_bias = np.random.default_rng()
rng_var = np.random.default_rng()

In [189]:
FACTOR_BIAS = {
    "Part": 35,
    "Reticle": 0.1,
    "Prev1Reticle": 0.05,
    "Prev2Reticle": 0.05,
    "Tool": 8,
    "Prev1Tool": 2,
    "Prev2Tool": 2,
    "ChuckID": 1,
}

In [190]:
def gen_part_name():
    _prefix = "TM"
    get_chr = lambda: chr(rng_part_name.integers(65,91))
    _suffix = f'{random.randint(0,99):02d}'
    return _prefix + get_chr() + get_chr() + _suffix

def gen_ret_names(part_name, layer_name, num):
    return [f'{part_name[2:6]}{layer_name}-{i}' for i in range(1, num+1)]

def gen_bias(feat, base=0):
    bias = rng_bias.random(size=[DIM,1]) - 0.5
    scale = 1 / np.array(LST_SCALE[:DIM]).reshape([DIM, -1])
    factor = FACTOR_BIAS[feat]
    return bias * scale * factor + base

In [191]:
@dataclass
class Part:
    id: int
    n_ret: list

    name: str=field(init=False)
    ret_name: dict=field(init=False)
    
    bias: list=field(init=False)
    ret_bias: dict=field(init=False)
    
    chuck_type: list=field(init=False)

    def __post_init__(self):
        self.name = gen_part_name()
        self.bias = gen_bias(feat="Part")
        self.ret_name = dict()
        self.ret_bias = dict()

        for lyr, lyr_name, num  in zip(LST_LAYER, LST_LAYER_NAME, self.n_ret):
            ret_names = gen_ret_names(self.name, lyr_name, num)
            self.ret_name[lyr] = ret_names

            for ret_name in ret_names:
                self.ret_bias[ret_name] = gen_bias(feat=lyr, base=self.bias)

        self.chuck_type = random.choice(LST_CHUCK )
        
    def __repr__(self):
        name = f'{"Part":<15}: {self.name}'
        reticle = "\n".join([f'{lyr:<15}: {self.ret_name[lyr]}' for lyr in LST_LAYER])
        chuck = f'{"Chuck Type":<15}: {self.chuck_type}'
        
        ret_bias = f'{"Reticle Bias":<15}:'
        for ret, bias in self.ret_bias.items():
            for idx, value in enumerate(bias):
                ret_bias += "\n"
                if idx == 0:
                    ret_bias += f'{ret:<9}: ['
                else:
                    ret_bias += f'{"":<12}'
                ret_bias += f'[{float(value):10.6f}]'
            ret_bias += "]"
        
        return name + "\n" + reticle + "\n" + chuck + "\n" + ret_bias


@dataclass
class Tool:
    id: int
    
    name: str=field(init=False)
    bias0: list=field(init=False)
    bias1: list=field(init=False)
    bias2: list=field(init=False)
    
    chuck0: list=field(init=False)
    chuck1: list=field(init=False)
    chuck2: list=field(init=False)
    
    def __post_init__(self):
        self.name = f'Tool-{self.id}'
        self.bias0 = gen_bias(feat="Tool")
        self.bias1 = gen_bias(feat="Prev1Tool")
        self.bias2 = gen_bias(feat="Prev2Tool")
        
        self.chuck0 = gen_bias(feat="Tool")
        self.chuck1 = gen_bias(feat="Prev1Tool")
        self.chuck2 = gen_bias(feat="Prev2Tool")

In [192]:
pp = Part(1, [2,1,2])
pp

Part           : TMMH93
Reticle        : ['MH93L0-1', 'MH93L0-2']
Prev1Reticle   : ['MH93L1-1']
Prev2Reticle   : ['MH93L2-1', 'MH93L2-2']
Chuck Type     : ['S1', 'S2']
Reticle Bias   :
MH93L0-1 : [[ 13.967317]
            [ 10.741389]
            [  0.002418]
            [  0.068371]
            [  0.040557]
            [ -0.030659]
            [  0.847663]
            [  0.550053]
            [  0.826884]
            [ -0.259549]]
MH93L0-2 : [[ 13.962378]
            [ 10.747391]
            [  0.002385]
            [  0.068321]
            [  0.041046]
            [ -0.031182]
            [  0.850178]
            [  0.546985]
            [  0.826244]
            [ -0.261778]]
MH93L1-1 : [[ 13.986105]
            [ 10.763489]
            [  0.002428]
            [  0.068421]
            [  0.040567]
            [ -0.030787]
            [  0.851007]
            [  0.548832]
            [  0.830347]
            [ -0.259486]]
MH93L2-1 : [[ 13.969940]
            [ 10.787385]
            

In [193]:
tt = Tool(1)
tt

Tool(id=1, name='Tool-1', bias0=array([[-1.497801  ],
       [ 0.68491536],
       [ 0.01625553],
       [ 0.02313736],
       [-0.02611924],
       [-0.0215547 ],
       [-0.14871374],
       [-0.0015875 ],
       [-0.24972751],
       [-0.05669308]]), bias1=array([[-0.37375679],
       [-0.02965811],
       [ 0.00414948],
       [ 0.00572594],
       [ 0.0063658 ],
       [ 0.00519698],
       [-0.02681201],
       [-0.02093143],
       [-0.01830393],
       [-0.04893658]]), bias2=array([[-9.11102973e-01],
       [-1.35703251e-01],
       [-4.75233716e-04],
       [-2.90173715e-03],
       [ 1.95516121e-03],
       [-6.23574432e-03],
       [-5.69404582e-02],
       [-7.17352522e-03],
       [ 2.89577577e-02],
       [ 1.01341174e-02]]), chuck0=array([[ 2.19227594e+00],
       [-1.10389496e+00],
       [-1.74275523e-02],
       [-8.87010548e-03],
       [-6.94327174e-04],
       [-2.02932785e-02],
       [-8.28266721e-02],
       [-1.12760701e-01],
       [-2.62857456e-01],
       [-

In [194]:
def gen_data(n_part, n_ret, n_tool):
    _part_map = [Part(i, n_ret) for i in range(n_part)]
    _tool_map = [Tool(i) for i in range(n_tool)]
    
    part_map = {part.name: part for part in _part_map}
    tool_map = {tool.name: tool for tool in _tool_map}
    return part_map, tool_map

n_part = 5
n_ret = [2,1,1]
n_tool = 10

part_map, tool_map = gen_data(n_part, n_ret, n_tool)

In [195]:
LST_FEAT = ["Part", "Reticle", "Prev1Reticle", "Tool", "Prev1Tool", "Prev2Tool", "ChuckID"]

lst_comb = []
for part_name, part in part_map.items():
    lst_tool = list(tool_map.keys())
    lst_feats = [[part_name], part.ret_name["Reticle"], part.ret_name["Prev1Reticle"], lst_tool[:5], lst_tool[5:], lst_tool[5:], part.chuck_type]
    lst_comb.extend(list(product(*lst_feats)))

df_data = pd.DataFrame(lst_comb, columns=LST_FEAT)
df_data


,Part,Reticle,Prev1Reticle,Tool,Prev1Tool,Prev2Tool,ChuckID
0,TMGH18,GH18L0-1,GH18L1-1,Tool-0,Tool-5,Tool-5,S1
1,TMGH18,GH18L0-1,GH18L1-1,Tool-0,Tool-5,Tool-5,S2
2,TMGH18,GH18L0-1,GH18L1-1,Tool-0,Tool-5,Tool-6,S1
3,TMGH18,GH18L0-1,GH18L1-1,Tool-0,Tool-5,Tool-6,S2
4,TMGH18,GH18L0-1,GH18L1-1,Tool-0,Tool-5,Tool-7,S1
...,...,...,...,...,...,...,...
2495,TMAR03,AR03L0-2,AR03L1-1,Tool-4,Tool-9,Tool-7,S2
2496,TMAR03,AR03L0-2,AR03L1-1,Tool-4,Tool-9,Tool-8,S1
2497,TMAR03,AR03L0-2,AR03L1-1,Tool-4,Tool-9,Tool-8,S2
2498,TMAR03,AR03L0-2,AR03L1-1,Tool-4,Tool-9,Tool-9,S1


In [196]:
def cal_rn(row):
    _Part = row["Part"]
    _Ret0 = row["Reticle"]
    _Ret1 = row["Prev1Reticle"]
    _Tool0 = row["Tool"]
    _Tool1 = row["Prev1Tool"]
    _Tool2 = row["Prev2Tool"]
    _Chuck = row["ChuckID"]
    
    rn0 = part_map[_Part].ret_bias[_Ret0][0] + tool_map[_Tool0].bias0[0]
    rn1 = part_map[_Part].ret_bias[_Ret1][0] + tool_map[_Tool1].bias1[0]
    rn2 = tool_map[_Tool2].bias2[0] + 0
    
    if _Chuck == "S1":
        rn0 += tool_map[_Tool0].chuck0[0]
        rn1 += tool_map[_Tool1].chuck1[0]
        rn2 += tool_map[_Tool2].chuck2[0]
    elif _Chuck == "S2":
        rn0 -= tool_map[_Tool0].chuck0[0]
        rn1 -= tool_map[_Tool1].chuck1[0]
        rn2 -= tool_map[_Tool2].chuck2[0]
        
    return float(rn0+rn1+rn2)
    
    
df_data["Tx"] = df_data.apply(cal_rn, axis=1)
df_data

,Part,Reticle,Prev1Reticle,Tool,Prev1Tool,Prev2Tool,ChuckID,Tx
0,TMGH18,GH18L0-1,GH18L1-1,Tool-0,Tool-5,Tool-5,S1,-3.649185
1,TMGH18,GH18L0-1,GH18L1-1,Tool-0,Tool-5,Tool-5,S2,-0.979781
2,TMGH18,GH18L0-1,GH18L1-1,Tool-0,Tool-5,Tool-6,S1,-2.379546
3,TMGH18,GH18L0-1,GH18L1-1,Tool-0,Tool-5,Tool-6,S2,-2.608577
4,TMGH18,GH18L0-1,GH18L1-1,Tool-0,Tool-5,Tool-7,S1,-2.894868
...,...,...,...,...,...,...,...,...
2495,TMAR03,AR03L0-2,AR03L1-1,Tool-4,Tool-9,Tool-7,S2,-6.024459
2496,TMAR03,AR03L0-2,AR03L1-1,Tool-4,Tool-9,Tool-8,S1,-9.440110
2497,TMAR03,AR03L0-2,AR03L1-1,Tool-4,Tool-9,Tool-8,S2,-8.454030
2498,TMAR03,AR03L0-2,AR03L1-1,Tool-4,Tool-9,Tool-9,S1,-10.772406


In [197]:
tool_map['Tool-4']

Tool(id=4, name='Tool-4', bias0=array([[ 3.27196853],
       [-2.01403718],
       [-0.01754182],
       [-0.02333162],
       [ 0.00723565],
       [-0.0062378 ],
       [ 0.2423258 ],
       [ 0.09078967],
       [ 0.21722401],
       [ 0.05790904]]), bias1=array([[-0.10116246],
       [-0.00585134],
       [ 0.00079169],
       [ 0.00580741],
       [ 0.00665466],
       [-0.00655152],
       [ 0.0234839 ],
       [-0.03824062],
       [ 0.06606712],
       [ 0.00633027]]), bias2=array([[-0.06056054],
       [-0.12442341],
       [ 0.00012888],
       [-0.0014737 ],
       [ 0.00627316],
       [-0.00023399],
       [ 0.03935467],
       [-0.02926087],
       [ 0.01196134],
       [-0.01918489]]), chuck0=array([[-2.08344122],
       [-0.87112498],
       [ 0.00917347],
       [-0.02629454],
       [-0.01727634],
       [ 0.00712743],
       [-0.2167196 ],
       [-0.204315  ],
       [-0.23615437],
       [ 0.1557893 ]]), chuck1=array([[ 0.36026376],
       [ 0.27817323],
       [ 0

In [198]:
import numpy as np

rng_part_name = np.random.default_rng()

rng_part_name.integers(0,3)

1

In [199]:
import numpy as np
rng = np.random.default_rng()

n_data = 501
n_part = 4
n_protect = 4

def _gen_nums(n_data, n_part, n_protect):
    lst_num = list()
    is_lt_protect = False
    while sum(lst_num) <= n_data or is_lt_protect:
        lst_num = rng.normal(loc=n_data/n_part, scale=n_data/n_part/2, size=n_part)
        lst_num = [int(x)>>1<<1 for x in lst_num]
        is_lt_protect = any([x<n_protect for x in lst_num])
    return lst_num

def gen_nums(n_data, n_part, n_protect=4):
    idx = 0
    lst_num = _gen_nums(n_data, n_part, n_protect)

    while sum(lst_num) >= n_data+2:
        if lst_num[idx%n_part] > n_protect:
            lst_num[idx%n_part] -= 2
        idx += 1

    return lst_num

gen_nums(n_data, n_part, n_protect)

[90, 128, 204, 80]

In [223]:
a = [1,5,7,9,3]

import random

rng.choice(a, size=5, replace=False)

array([1, 3, 5, 7, 9])